** link to the github : https://github.com/AxeldeRomblay/MLBox **


# Inputs & imports : that's all you need to give !


In [1]:
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *

In [2]:
paths = ["../input/train.json", "../input/test.json"]
target_name = "interest_level"


# Now let MLBox do the job !


## ... to read and clean all the files

In [3]:
rd = Reader()
df = rd.train_test_split(paths, target_name)

In [4]:
df["train"].head()

In [5]:
dft = Drift_thresholder()
df = dft.fit_transform(df)

## ... to evaluate a default configuration

In [7]:
opt = Optimiser()

In [8]:
opt.evaluate(None, df)

## ... or to tune hyper parameters 

In [9]:
space = {
    
        "ne__numerical_strategy" : {"search":"choice", "space":["mean", 0]},  
    
        "ce__strategy" : {"search":"choice", "space":["label_encoding", "random_projection"]},
    
        'est__strategy':{"search":"choice", "space":["LightGBM"]},    
        'est__max_depth':{"search":"choice", "space":[6,7]},
        'est__learning_rate':{"search":"uniform", "space":[0.01, 0.03]},
        'est__subsample':{"search":"uniform", "space":[0.8, 0.9]},
        'est__colsamplebytree':{"search":"uniform", "space":[0.7, 0.8]}
    
        }

params = opt.optimise(space, df, 2)

You can  tune the whole Pipeline ! Indeed, you can choose:

- different strategies to impute missing values
- different strategies to encode categorical features (entity embeddings, ...)
- different strategies and thresholds to select relevant features (random forest feature importance, l1 regularization, ...)
- to add stacking meta-features !
- different models and hyper-parameters (XGBoost, Random Forest, Linear, ...)



## ... to predict

In [10]:
prd = Predictor()
prd.fit_predict(params, df)

### Formatting for submission¶


In [11]:
submit = pd.read_csv("save/"+target_name+"_predictions.csv")[["high", "medium", "low"]]
submit["listing_id"] = df["test"].listing_id.astype(int).values
submit.to_csv("mlbox.csv", index=False)

BONUS : stacking

In [12]:
stck = {
    "stck__base_estimators" : [
        Classifier(strategy="XGBoost"),
        Classifier(strategy="RandomForest"),
        Classifier(strategy="LightGBM")
    ],
    "est__strategy" : "RandomForest",
    "est__max_depth" : 3,
    "est__max_features" : 1.
}

In [13]:
# prd.fit_predict(stck, df)

In [14]:
# submit = pd.read_csv("save/"+target_name+"_predictions.csv")[["high", "medium", "low"]]
# submit["listing_id"] = df["test"].listing_id.astype(int).values
# submit.to_csv("mlbox_stck.csv", index=False)